# MANVUE Image Setup for Google Colab

This notebook demonstrates how to convert external images to MongoDB GridFS using MANVUE's API.

## Prerequisites
1. Your MANVUE backend server must be running
2. MongoDB must be accessible from your Colab environment
3. Upload the `colab_image_setup.py` script to your Colab environment


In [ ]:
# Step 1: Upload the colab_image_setup.py script to your Colab environment
# You can do this by:
# 1. Using the file upload button in Colab
# 2. Or cloning from your repository
# 3. Or copying the script content directly

print("📁 Please upload colab_image_setup.py to your Colab environment")


In [ ]:
# Step 2: Import the setup functions
from colab_image_setup import run_colab_setup, convert_all_product_images_colab
import asyncio

print("✅ Imported MANVUE Colab setup functions")


In [ ]:
# Step 3: Configure your API URL
# Replace with your actual MANVUE API URL
API_URL = "http://localhost:5001"  # Local development
# API_URL = "https://your-deployed-api.com"  # Deployed API

print(f"🔗 Using API URL: {API_URL}")


In [ ]:
# Step 4: Run the complete setup
# This will download images and store them in your MongoDB GridFS

converted_products = run_colab_setup(API_URL)

if converted_products:
    print(f"\n🎉 Successfully converted {len(converted_products)} products!")
    print("Your images are now stored in MongoDB GridFS.")
else:
    print("\n❌ Setup failed. Please check your API connection.")
